<a href="https://colab.research.google.com/github/jackevansadl/CHEM3630/blob/main/workshop3-thermodynamicsA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A. Autoionisation of Water
Combined geometry optimisation and vibrational frequency calculations have been performed for water and its conjugate base, the hydroxide ion, using DFT (at the _B3LYP/6-31+G(d)_ level of theory). Use the results of these calculations to calculate the change in energy, standard enthalpy, and standard Gibbs free energy for the gas- phase acid ionisation of water at 298.15 K (25$^\circ$C).

First we need to install the DFT codes (this will take a few minutes)

In [2]:
!pip install -q condacolab

In [3]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [4]:
! mamba install -c anaconda intel-openmp --quiet >/dev/null 2>&1
! mamba install -c psi4 psi4 --quiet >/dev/null 2>&1
! pip install -q ase >/dev/null 2>&1
! pip install -q nglview >/dev/null 2>&1
! pip install -q 'ipywidgets>=7.6.0,<8' --force-reinstall >/dev/null 2>&1

The _ase_ (atomic simulation environment) library has routines for describing molecules and interfaces with DFT software.

In [5]:
from ase.build import molecule
from ase import Atoms

water = molecule('H2O')

d=0.97
ohminus = Atoms('OH', positions=([0-d/2,0,0],[0+d/2,0,0]), charges=[-1,0])

hplus = molecule('H')
hplus.set_initial_charges([+1])

molecules = [water, ohminus]

In [18]:
#initalise dictionaries to save the results below
h_results = {}
oh_results = {}
h2o_results = {}

Computing the energy for H$^{+}$ is doesn't require any geometry optimisation (it's only one atom).

The calculation parameters are loaded with the Psi4 object. Check out https://wiki.fysik.dtu.dk/ase/ase/calculators/psi4.html for the full list of options. **There is an error in the parameters below!**

In [32]:
from ase.calculators.psi4 import Psi4
calc = Psi4(atoms = hplus,
            method = 'b3lyp',
            memory = '500MB', # this is the default, be aware!
            basis = '6-31+G(d)',
            charge = +1,
            multiplicity=1,
            label = hplus.get_chemical_formula())

hplus.calc = calc
print(hplus.get_potential_energy())
h_results['energy'] = hplus.get_potential_energy()

/usr/local/lib/python3.8/site-packages/ase/calculators/psi4.py:116: UserWarning: A charge was provided without a spin multiplicity. A multiplicity of 1 is assumed
  warnings.warn('A charge was provided without a spin '


0.0


__What is the energy for H$^+$ and in which units?__

 Answer:

For molecules we need to first optimise the atom positions to find the lowest energy positons. The calculation optimises the molecular structure through a series of geometry optimisation steps. Search for "Step number" to determine the number of steps to reach convergence.

In [33]:
from ase.calculators.psi4 import Psi4
from ase.build import molecule
import numpy as np
from ase.optimize import QuasiNewton

optimised_molecules = []
for atoms in molecules:
  print(atoms.get_chemical_formula())
  calc = Psi4(atoms = atoms,
        method = 'b3lyp',
        memory = '500MB', # this is the default, be aware!
        basis = '6-31+G(d)',

        charge = np.sum(atoms.get_initial_charges()),
        multiplicity=1,
        label = atoms.get_chemical_formula())

  atoms.calc = calc

  QuasiNewton(atoms).run(fmax=0.0005)

  #save results
  if atoms.get_chemical_formula() == 'HO':
      oh_results['energy'] = atoms.get_potential_energy()
  if atoms.get_chemical_formula() == 'H2O':
      h2o_results['energy'] = atoms.get_potential_energy()
  print(atoms.get_potential_energy())
  optimised_molecules.append(atoms)

H2O
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 04:39:34    -2079.564177        0.0005
-2079.5641771711976
HO
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 04:39:38    -2062.532703        0.0000
-2062.5327029101454


 __What is the convergence criteria used here?__

 Answer:

  __What optimiser is used here and what alternatives are there?__

 Answer:

The output displayed is only a snapshot of important quantities. The files _H.dat_, _HO.dat_ and _H20.dat_ have the raw output from the _Psi4_ sofware.

__Open the *H2O.dat* file and search the detailed ouput. Which point group symmetry is used here?__

Answer:

Using the harmonic oscilator approximation the vibrational degrees of freedom (normal modes) of these molecules can be computed.

In [12]:
from ase.vibrations import Vibrations
#save the results in a list for later processing
vibs = []
for atoms in optimised_molecules:
  print(atoms.get_chemical_formula())
  vib = Vibrations(atoms, name=atoms.get_chemical_formula(), delta=0.01)
  vib.clean()
  vib.run()
  vib.write_mode(0)
  vib.write_mode(-1)
  vib.summary()
  vibs.append(vib)


H2O
---------------------
  #    meV     cm^-1
---------------------
  0    5.9i     47.4i
  1    0.3i      2.8i
  2    0.2i      1.4i
  3    0.1       0.8
  4    3.3      26.8
  5    4.8      38.7
  6  206.0    1661.8
  7  463.2    3735.8
  8  478.6    3860.1
---------------------
Zero-point energy: 0.578 eV
HO
---------------------
  #    meV     cm^-1
---------------------
  0    0.0i      0.0i
  1    0.0i      0.0i
  2    0.0i      0.0i
  3    3.3      26.7
  4    3.3      26.7
  5  453.3    3656.3
---------------------
Zero-point energy: 0.230 eV


The modes as displayed include translational and rotational movements that leads low energy negative frequences, these can be ignored.

__(In general) What do negative frequencies suggest?__

Answer:

The trajectory of the 0th mode was saved (_H2O.0.traj_) and you can view this using the _view_ routine in _ase__:

In [31]:
from google.colab import output
output.enable_custom_widget_manager()

from ase.visualize import view
from ase.io import read
atoms = read('/content/H2O.0.traj', index=":")
view(atoms, viewer='ngl')

__Confirm that the 8th vibrational mode of H2O and the 5th mode of HO relate to bond stretching__.
_You may need to clear the output of the above cell_

In [ ]:
# your code goes here

The _thermochemistry_ routines of ASE can be used to compute several thermodynamic quantites from the normal modes using the ideal gas approximation. An example for H2O is shown below: 

In [30]:
from ase.thermochemistry import IdealGasThermo
# the 0th entry is h2o
atoms = optimised_molecules[0]
potentialenergy = atoms.get_potential_energy()
vib = vibs[0]
vib_energies = vib.get_energies()


thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=atoms,
                        geometry='linear',
                        symmetrynumber=2, spin=0)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)
H = thermo.get_enthalpy(temperature=298.15, verbose=False)
S = thermo.get_entropy(temperature=298.15, pressure=101325., verbose=False)
h2o_results['gibbs_energy'] = G
h2o_results['enthalpy'] = H
h2o_results['entropy'] = S

Enthalpy components at T = 298.15 K:
E_pot              -2079.564 eV
E_ZPE                  0.576 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.026 eV
Cv_vib (0->T)          0.023 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                  -2078.875 eV

Entropy components at T = 298.15 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 bar)    0.0015019 eV/K        0.448 eV
S_rot              0.0002940 eV/K        0.088 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0002312 eV/K        0.069 eV
S (1 bar -> P)    -0.0000011 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0020260 eV/K        0.604 eV

Free energy components at T = 298.15 K and P = 101325.0 Pa:
    H      -2078.875 eV
 -T*S         -0.604 eV
-----------------------
    G      -2079.479 eV


__Confirm that the average translation energy (described here as translational heat capacity Cv) is described by Eq. 9 (Appendix):__

$<ɛ_T> = -\frac{1}{q_T}(\frac{∂q_T}{∂β})_{N,V} \approx \frac{3}{2}k_BT$

In [ ]:
#answer goes here

__Compute the same thermodynamic quanties (gibbs energy, enthalpy and entropy) for OH and H$^+$.__

In [ ]:
#answer for OH

In [ ]:
#answer for H+

At this point you should have three dictionaries _h\_results_, _oh\_results_ and _h2o\_results_ that includes the electronic energy, enthalpy and entropy of these species.

__Calculate the change in internal energy $\Delta$U, the change in enthalpy $\Delta$H, the entropy contribution T$\Delta$S, and the Gibbs free energy change $\Delta$G for the reaction (Eqns 23-27):__

H$_2$O $⟶$ OH$^{-}$ + H$^+$ 


In [ ]:
#answer for deltaU

In [ ]:
#answer for deltaH

In [ ]:
#answer for deltaS

In [ ]:
#answer for deltaG

__Calculate K$_a$ and pK$_a$ at 298.15 K for the acid-ionisation reaction (H2O ⇌ H$^+$ + OH$^-$). Discuss how this compares with the experimentally determined value and why it might differ.__

In [ ]:
#answer for Ka

In [ ]:
#answer for pKa

_Your discussion_